In [3]:
import random
import time
import threading
import sys
import select
from tqdm import tqdm
from selenium import webdriver
from _config import ScraperConfig
from _crawler_helper_func import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [6]:
options = webdriver.ChromeOptions()
options.add_argument("--headless") 
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

crawler  = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), 
    options=options
)

In [7]:
sc = ScraperConfig()

In [8]:
def write_file(llinks):
    with open("crawaler.txt", "w") as f:
        for link in llinks:
            f.write(link + "\n")

def read_file():
    try:
        with open('crawaler.txt', 'r') as f:
            file = f.read()
            file_exits = (len(file) != 0)
            return file_exits, file.split('\n')
    except:
        file_exits = False
        return file_exits, None

In [9]:
def rest_for_min(minu=2, pos=0):
    print("\nStarting to rest---[Enter to skip/ Ctrl + C to canel]")
    minu = int(minu * 60)
    stop_flag = {"skip": False, "done": False}
    
    def wait_for_enter(stop_flag):
        while not stop_flag["done"]:
            if sys.stdin in select.select([sys.stdin], [], [], 0.1)[0]:
                input() 
                stop_flag["skip"] = True
                return
        
    t = threading.Thread(target=wait_for_enter, args=(stop_flag,), daemon=True)
    t.start()
    try:
        pbar = tqdm(range(minu), desc="Resting.......", position=pos)
        for i in pbar:
            if stop_flag["skip"]:
                pbar.set_description('Skipping.......[Enter Pressed!]')
                break
            remaining = minu - i
            pbar.set_description(f"Resting... {remaining}s left")
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nExiting the resting phase")

In [20]:
state, links = read_file()

if not state:
    links = [sc.default_link]

In [29]:
def crawl(links, crawler, return_=True, stop_after_min=0.06):
    pbar = tqdm(range(sc.iterations), total=sc.iterations, desc='Starting crawling....', position=0)
    links = set(links)
    start = 0
    isRest = False

    def check_min():
        nonlocal start, isRest
        while True:
            start += 1
            if start % (stop_after_min * 60) == 0:
                isRest = True
            time.sleep(1)

    t = threading.Thread(target=check_min, daemon=True)
    t.start()
    
    for itr in pbar:
        sl = random.uniform(1, 2)
        pbar.set_description(f'Sleeping for {sl:.1f}')
        time.sleep(sl)
    
        try:
            pbar.set_description('Calling crawler....')
            crawler.get(links.pop())
            
            pbar.set_description('Getting Links from Page....')
            clinks = crawler.find_elements(By.XPATH, '//a[@href]')
            time.sleep(2)
            
            pbar.set_description('Filtering Links....')
            for l in clinks:
                try:
                    link = l.get_attribute('href')
                    if ('/learn' in link or '/browse' in link or '/courses' in link or '/explore' in link):
                        if ('/articles' not in link and '?authMode' not in link and '//chat' not in link):
                            links.add(link)
                except Exception as e:
                    pbar.set_description(f'Error: {type(e).__name__}')
            
            write_file(list(links))
        except:
            pbar.set_description('Something went wrong...Must check the connection')
    

        if isRest:
            isRest = False
            try:
                rest_for_min()
            except:
                pbar.set_description('Error in resting')

    if return_:
        return links

In [30]:
crawl(links, crawler)

Filtering Links....: 100%|██████████████████████████████████████| 2/2 [00:21<00:00, 10.74s/it]


{'https://learner.coursera.help/hc',
 'https://www.coursera.org/browse',
 'https://www.coursera.org/browse/arts-and-humanities',
 'https://www.coursera.org/browse/business',
 'https://www.coursera.org/browse/computer-science',
 'https://www.coursera.org/browse/data-science',
 'https://www.coursera.org/browse/information-technology',
 'https://www.coursera.org/browse/language-learning',
 'https://www.coursera.org/browse/life-sciences',
 'https://www.coursera.org/browse/math-and-logic',
 'https://www.coursera.org/browse/personal-development',
 'https://www.coursera.org/browse/physical-science-and-engineering',
 'https://www.coursera.org/browse/social-sciences',
 'https://www.coursera.org/courses?query=artificial%20intelligence',
 'https://www.coursera.org/courses?query=big%20data',
 'https://www.coursera.org/courses?query=business&topic=Business',
 'https://www.coursera.org/courses?query=computer%20science&topic=Computer%20Science',
 'https://www.coursera.org/courses?query=cybersecurity'